In [ ]:
#Tipical pubchem html structure

#Title: 
#<head><title> str

#GHStable:
#<body>
#<div class="theme-compound" id="root">
#<div class="relative min-h-screen flex flex-col">flex
#<main id="main-content" class="print:border-0 border-b-4 border-theme flex-1">
#<div class="main-width app-wrapper">
#<div class="lg:flex lg:flex-nowrap">
#<div class="lg:w-2/3 2xl:w-3/4 space-y-4">
#<div id="section-container" class="py-8">
#<section id="Safety-and-Hazards" class="pt-4">
#<section id="Hazards-Identification" class="pt-4"> 
#<section id="GHS-Classification" class="pt-4">
#<div class="px-1 py-3 space-y-2">
#<div class>
#<div class="sm:table w-full">
#<div class="sm:table-row-group">

#GHS Hazard Statements:
#<div class="p-2 sm:table-row pc-gray-border-t sm:border-0 ">…</div>
#<div class="text-left sm:table-cell sm:align-middle sm:p-2  sm:border-t sm:border-gray-300 dark:sm:border-gray-300/20  pb-1 pl-2">
#<div class="break-words space-y-1">
#<p>

#Precautionary Statement Codes:
#<div class="p-2 sm:table-row pc-gray-border-t sm:border-0 ">…</div>
#<div class="text-left sm:table-cell sm:align-middle sm:p-2  sm:border-t sm:border-gray-300 dark:sm:border-gray-300/20  pb-1 pl-2">
#<div class="break-words space-y-1">
#<p>

In [29]:
import requests
from bs4 import BeautifulSoup

pubchem_url = "https://pubchem.ncbi.nlm.nih.gov/compound/2244"

# Fetch the content of the webpage
response = requests.get(pubchem_url)
response.raise_for_status()

#Parsing
soup = BeautifulSoup(response.content, 'html.parser')

root = soup.find('div', "theme-compound", "root")
print(root)

# DEAD end, that is due to the content being dynamically loaded using JavaScript
#This is common with moder web applications that use frameworks like React, Angular, or Vue.js

<div class="theme-compound" id="root"></div>


In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

chrome_options = Options()
chrome_options.add_argument("--headless") #no GUI

driver = webdriver.Chrome()
driver.get(pubchem_url)
time.sleep(5)
page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html.parser')
root = soup.find('div', "theme-compound", "root")
print(root)

driver.quit()

None


In [60]:
#print(soup.find_all('div', 'break-words space-y-1', 'p'))
index = ['H302', 'H315', 'H319']
HazardStat = soup.find_all(string=lambda text: any(s in text for s in index))
for elem in HazardStat:
    print(elem)

H302 (95.27%): Harmful if swallowed [
H315 (21.62%): Causes skin irritation [
H319 (23.31%): Causes serious eye irritation [


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from io import open
import time


def dictCodeList(path="dictionary/ghscode_10.txt"):
    
    codigos = []
    with open('dictionary/ghscode_10.txt', 'r') as fichero:
        for linea in fichero:
            lista = linea.split()
            if not linea.isspace():
                codigos.append(lista[0])
    return codigos

def GHScodefinder(pubchem_url=None):

    """
    The chemGHSclass func will return the GHS classification of any chemical published in Pubchem
    by scrapping the website with selenium and comparing the codes found within its html source 
    with the dictionary database
    """
    
    chrome_options = Options()
    chrome_options.add_argument("--headless") #no GUI

    driver = webdriver.Chrome()
    driver.get(pubchem_url)
    time.sleep(5) #Waits for the page to load completely
    page_source = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(page_source, 'html.parser')

    dict_list = dictCodeList()
    HazardStat = soup.find_all(string=lambda text: any(s in text for s in dict_list))
    
    codes_found = []
    for large_string in HazardStat:
        nocommas = large_string.replace(",", " ")
        listOfstrings = nocommas.split()
        for smaller_strings in listOfstrings:
            if smaller_strings in codigos:
                codes_found.append(smaller_strings)
    return codes_found

result = GHScodefinder("https://pubchem.ncbi.nlm.nih.gov/compound/2244")

TypeError: argument of type 'CodeDict' is not iterable

In [4]:
print(result)

NameError: name 'result' is not defined

In [5]:
from io import open

class CodeDict:

    #The default path uses backslashes which can cause issues on some systems. 
    #Using raw strings or forward slashes is a better approach.
    def __init__(self, path="dictionary/ghscode_10.txt"):
        self.code = []
        self.state = []
        self.clase = []
        self.cate = []
        self.div = []
        self.picto = []
        self.signal_p = []
        with open(path, 'r') as fichero:
            for line in fichero:
                if line.strip():
                    parts = line.split("\t")
                    self.code.append(parts[0])
                    self.state.append(parts[1])
                    self.clase.append(parts[2])
                    self.cate.append(parts[3])
                    self.div.append(parts[4])
                    self.picto.append(parts[5])
                    self.signal_p.append(parts[6])

codigos = CodeDict()
print(codigos.cate)

['GHS Hazard Category', 'Unstable Explosive', 'Div 1.1', 'Div 1.2', 'Div 1.3', 'Category 2B', 'Category 2C', 'Div 1.5', 'Div 1.6', 'Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 1', 'Category 2A', 'Category 1', 'Category 1', '1A: Flammable gas, Pyrophoric gas, Chemically unstable gas A,B', '1B', 'Category 2', 'Category 1', 'Category 2', 'Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 1', 'Category 2', 'Category 1', 'Category 2', 'Category 3', '1A, Chemically unstable gas A', '1A, Chemically unstable gas B', '1A, Pyrophoric gas', 'Type A', 'Type B', 'Type C, D', 'Type E, F', 'Type G', 'Category 1', 'Category 1', 'Category 1', 'Category 2', 'Category 1', 'Category 2', 'Category 3', 'Category 1', 'Category 1', 'Category 2', 'Category 3', 'Compressed gas, Liquefied gas, Dissolved gas', 'Refrigerated liquefied gas', 'Category 1', 'Category 2', 'Category 3', 'Category 1', 'Category 1, 2', 'Category 3', 'Category 4', 'Category 5', 'Category 1', 'Category 2

In [56]:
#All of the codes are 4 characters long, some of them are fused with a '+' sign
#I want to create a list of only codes.
#Algorithm list of codes
#Input: a list of strings which contain 4 letter codes within them, sometimes they are longer when they
#       followed by a '+' or have more than one code
#Output: a list of formated strings with only the 4 character codes whitin them or the fused versions


codes_found = []
for large_string in result:
    nocommas = large_string.replace(",", " ")
    listOfstrings = nocommas.split()
    for smaller_strings in listOfstrings:
        if smaller_strings in codigos.code:
            codes_found.append(smaller_strings)
print(codes_found)

['H302', 'H315', 'H319', 'H335', 'P261', 'P264', 'P264+P265', 'P270', 'P271', 'P280', 'P301+P317', 'P302+P352', 'P304+P340', 'P305+P351+P338', 'P319', 'P321', 'P330', 'P332+P317', 'P337+P317', 'P362+P364', 'P403+P233', 'P405', 'P501']


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from io import open
import time


class CodeDict:

    #The default path uses backslashes which can cause issues on some systems. 
    #Using raw strings or forward slashes is a better approach.
    def __init__(self, path="dictionary/ghscode_10.txt"):
        self.code = []
        self.state = []
        self.clase = []
        self.cate = []
        self.div = []
        self.picto = []
        self.signal_p = []
        with open(path, 'r') as fichero:
            for line in fichero:
                if line.strip():
                    parts = line.split("\t")
                    self.code.append(parts[0])
                    self.state.append(parts[1])
                    self.clase.append(parts[2])
                    self.cate.append(parts[3])
                    self.div.append(parts[4])
                    self.picto.append(parts[5])
                    self.signal_p.append(parts[6])

def chemGHScode(pubchem_url=None):

    """
    The chemGHSclass func will return the GHS classification of any chemical published in Pubchem
    by scrapping the website with selenium and comparing the codes found within its html source 
    and the local dictionary database
    """
    
    chrome_options = Options()
    chrome_options.add_argument("--headless") #no GUI

    driver = webdriver.Chrome()
    driver.get(pubchem_url)
    time.sleep(0.5) #Waits for the page to load completely
    page_source = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(page_source, 'html.parser')

    code_dict = CodeDict()
    HazardStat = soup.find_all(string=lambda text: any(s in text for s in code_dict.code))

    codes_found = []
    for large_string in HazardStat:
        nocommas = large_string.replace(",", " ")
        listOfstrings = nocommas.split()
        for smaller_strings in listOfstrings:
            if smaller_strings in code_dict.code:
                codes_found.append(smaller_strings)
    return codes_found



result = chemGHScode("https://pubchem.ncbi.nlm.nih.gov/compound/2244")
print(result)

['H302', 'H315', 'H319', 'H335', 'P261', 'P264', 'P264+P265', 'P270', 'P271', 'P280', 'P301+P317', 'P302+P352', 'P304+P340', 'P305+P351+P338', 'P319', 'P321', 'P330', 'P332+P317', 'P337+P317', 'P362+P364', 'P403+P233', 'P405', 'P501']


In [24]:
class labelBuilder():
    code_dict = CodeDict()

    def __init__(self,codes_found):
        self.dict_cont = {}
        code_dict = CodeDict()
        for code in codes_found:
            for index, ref_code in enumerate(code_dict.code):
                if code == ref_code:
                    self.dict_cont[code] = [
                        code_dict.state[index], 
                        code_dict.clase[index], 
                        code_dict.cate[index], 
                        code_dict.div[index],
                        code_dict.picto[index],
                        code_dict.signal_p[index],
                        ]
        """self.prod_id = []
        self.sign_w = []
        self.hazard_stat = []
        self.prec_stat = []
        self.supp_info = []
        self.picto = []"""

In [25]:
label = labelBuilder(result)
label.dict_cont

['H302', 'H315', 'H319', 'H335', 'P261', 'P264', 'P264+P265', 'P270', 'P271', 'P280', 'P301+P317', 'P302+P352', 'P304+P340', 'P305+P351+P338', 'P319', 'P321', 'P330', 'P332+P317', 'P337+P317', 'P362+P364', 'P403+P233', 'P405', 'P501']


{'H302': ['Harmful if swallowed',
  'Acute toxicity, oral',
  'Category 4',
  '',
  'GHS07',
  'Warning'],
 'H315': ['Causes skin irritation',
  'Skin corrosion/irritation',
  'Category 2',
  '',
  'GHS07',
  'Warning'],
 'H319': ['Causes serious eye irritation',
  'Serious eye damage/eye irritation',
  'Category 2A',
  '',
  'GHS07',
  'Warning'],
 'H335': ['May cause respiratory irritation',
  'Specific target organ toxicity, single exposure; Respiratory tract irritation',
  'Category 3',
  '',
  'GHS07',
  'Warning'],
 'P261': ['Avoid breathing dust/fume/gas/mist/vapors/spray.',
  '',
  '',
  '',
  '',
  ''],
 'P264': ['Wash hands [and ...] thoroughly after handling.',
  '',
  '',
  '',
  '',
  ''],
 'P264+P265': ['Wash hands [and ...] thoroughly after handling. Do not touch eyes.',
  '',
  '',
  '',
  '',
  ''],
 'P270': ['Do not eat, drink or smoke when using this product.',
  '',
  '',
  '',
  '',
  ''],
 'P271': ['Use only outdoors or in a well-ventilated area.',
  '',
  '',
  '